# import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc

In [ ]:
# load CSV's
df_train = pd.read_csv('/home/malmason/datasets/siim-isic-melanoma-classification/cleaned_csv/train_pp.csv')
df_test = pd.read_csv('/home/malmason/datasets/siim-isic-melanoma-classification/cleaned_csv/test_pp.csv')

In [ ]:
# Point to image folders
img_train_folder = '/home/malmason/datasets/siim-isic-melanoma-classification/ycbcr/norm/train/'
img_test_folder = '/home/malmason/datasets/siim-isic-melanoma-classification/ycbcr/norm/test/'

In [ ]:
df_train_csv = df_train[:26000]
df_val_csv   = df_train[26000:]

In [ ]:
df_train_csv = df_train_csv.append(df_train_csv.loc[df_train_csv['target'] == 1])
df_train_csv = df_train_csv.append(df_train_csv.loc[df_train_csv['target'] == 1])

In [ ]:
df_train_csv = df_train_csv.sample(frac=1).reset_index(drop=True)

In [ ]:
df_train_csv.shape

In [ ]:
# Used for looping through image columns
X_train_img = df_train_csv['image_name']
X_val_img = df_val_csv['image_name']

In [ ]:
X_train_csv = df_train_csv.drop(['target', 'image_name'], axis=1).values
y_train_csv = df_train_csv['target'].values
X_val_csv = df_val_csv.drop(['target', 'image_name'], axis=1).values
y_val_csv = df_val_csv['target'].values

In [ ]:
print(X_train_csv.shape, y_train_csv.shape, X_val_csv.shape, y_val_csv.shape)

In [ ]:
sizing = 224 - 2            # conv1
sizing = (sizing - 2) / 2   # conv2  
sizing = (sizing - 2)       # conv3
sizing = (sizing - 2) / 2   # conv4
sizing = (sizing - 2)       # conv5
sizing = (sizing - 2) / 2   # conv6
sizing = (sizing - 2)       # conv7
sizing = (sizing - 2) / 2   # conv8
sizing = (sizing - 2) /2    # conv9
print(512*(int(sizing)*int(sizing)))

In [ ]:
sizing = 224 - 4            # conv1
sizing = (sizing - 4) / 2   # conv2  
sizing = (sizing - 4) / 2   # conv4
sizing = (sizing - 4) / 2   # conv6
sizing = (sizing - 4) / 2   # conv8
sizing = (sizing - 4) / 2   # conv9
print(512*(int(sizing)*int(sizing)))

In [ ]:
class CNN(nn.Module):
  def __init__(self, n_output_neurons):
    super(CNN, self).__init__()

    self.conv1 = nn.Conv2d(3 , 32,   kernel_size=3, stride=1)
    self.conv2 = nn.Conv2d(32, 32,   kernel_size=3, stride=1)
    self.bn1   = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32 , 64,  kernel_size=3, stride=1)
    self.conv4 = nn.Conv2d(64 , 64,  kernel_size=3, stride=1)

    self.conv5 = nn.Conv2d(64 , 128, kernel_size=3, stride=1)
    self.conv6 = nn.Conv2d(128 ,128, kernel_size=3, stride=1)

    self.conv7 = nn.Conv2d(128, 256, kernel_size=3, stride=1)
    self.conv8 = nn.Conv2d(256 ,256, kernel_size=3, stride=1)
    self.bn4   = nn.BatchNorm2d(256)
    self.conv9 = nn.Conv2d(256, 512, kernel_size=3, stride=1)


    self.conv11 = nn.Conv2d(3 , 32,   kernel_size=5, stride=1)
    self.bn11   = nn.BatchNorm2d(32)
    self.conv21 = nn.Conv2d(32 , 64,  kernel_size=5, stride=1)

    self.conv31 = nn.Conv2d(64 , 128, kernel_size=5, stride=1)

    self.conv41 = nn.Conv2d(128, 256, kernel_size=5, stride=1)

    self.conv51 = nn.Conv2d(256, 512, kernel_size=5, stride=1)
    self.bn51   = nn.BatchNorm2d(512)
    
    self.data1 = nn.Linear(15, 128)
    self.data2 = nn.Linear(128, 16)
    
    self.img_fc1   = nn.Linear(8192+4608,128)
    self.img_fc2   = nn.Linear(128,32)
    self.img_fc3   = nn.Linear(32,16)
    
    self.fc1_conc  = nn.Linear(16,n_output_neurons)

  def forward(self, x, xd):
    
    xa = F.relu(self.conv1(x),2)
    xa = F.relu(F.max_pool2d(self.conv2(xa),2))
    xa = self.bn1(xa)
    xa = F.relu(self.conv3(xa),2)
    xa = F.relu(F.max_pool2d(self.conv4(xa),2))

    xa = F.relu(self.conv5(xa),2)
    xa = F.relu(F.max_pool2d(self.conv6(xa),2))

    xa = F.relu(self.conv7(xa),2)
    xa = F.relu(F.max_pool2d(self.conv8(xa),2))
    xa = self.bn4(xa)
    xa = F.relu(F.max_pool2d(self.conv9(xa),2))


    xb = F.relu(F.max_pool2d(self.conv11(x),2))
    xb = self.bn11(xb)
    xb = F.relu(F.max_pool2d(self.conv21(xb),2))

    xb = F.relu(F.max_pool2d(self.conv31(xb),2))

    xb = F.relu(F.max_pool2d(self.conv41(xb),2))

    xb = F.relu(F.max_pool2d(self.conv51(xb),2))
    xb = self.bn51(xb)
    
    xa = xa.view(xa.size(0), -1)
    xb = xb.view(xb.size(0), -1)
    
    xi = torch.cat((xa, xb), dim=1)
    
    xi = F.leaky_relu(self.img_fc1(xi))
    xi = F.dropout(xi, p=0.3)
    
    xi = F.leaky_relu(self.img_fc2(xi))
    xi = F.dropout(xi, p=0.2)
    xi = F.leaky_relu(self.img_fc3(xi))
    
    xd = F.leaky_relu(self.data1(xd))
    xd = F.dropout(xd, p=0.2)
    xd = F.leaky_relu(self.data2(xd))
    
    x = torch.cat((xi, xd), dim=1)
    
    x = self.fc1_conc(xd)

    return x

In [ ]:
n_output_neurons = 1

In [ ]:
model = CNN(n_output_neurons)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

In [ ]:
X_train_image = []
for image_get in X_train_img:
    img_train = cv2.imread(img_train_folder + '{}.jpg'.format(image_get))
    '''
    rand_num = np.random.uniform(low=1, high=1.2)
    M = cv2.resize(img_train, None, fx= rand_num, fy= rand_num, interpolation= cv2.INTER_LINEAR)
    
    H_crop = ((224*rand_num)-224)/2
    V_crop = ((224*rand_num)-224)/2
    
    C = M[np.int(H_crop):np.int(M.shape[0]-H_crop),np.int(V_crop):np.int(M.shape[1]-np.int(V_crop))]
    img_train= cv2.resize(C,(224,224))
    '''
    X_train_image.append(img_train)

In [ ]:
X_val_image = []
for image_get in X_val_img:
    img_val = cv2.imread(img_train_folder + '{}.jpg'.format(image_get))
    
    X_val_image.append(img_val)

In [ ]:
X_train_image = np.array(X_train_image)
X_train_image = X_train_image.astype(np.float32)

In [ ]:
X_val_image = np.array(X_val_image)
X_val_image = X_val_image.astype(np.float32)

In [ ]:
Y_train = np.array(y_train_csv)
Y_val = np.array(y_val_csv)

In [ ]:
X_train_image = X_train_image / 255
X_val_image = X_val_image / 255

In [ ]:
print(f'Train shape: {X_train_image.shape}, Vl shape: {X_val_image.shape}')

In [ ]:
X_train_image_t = np.transpose(X_train_image, (0,3,1,2))

In [ ]:
input_train = torch.from_numpy(X_train_image_t)

In [ ]:
target_train = torch.from_numpy(Y_train).reshape(-1,1).float()

In [ ]:
X_val_image_t = np.transpose(X_val_image, (0,3,1,2))

In [ ]:
input_val = torch.from_numpy(X_val_image_t)

In [ ]:
target_val = torch.from_numpy(Y_val).reshape(-1,1).float()

In [ ]:
X_train_csv = X_train_csv.astype(np.float32)
X_val_csv   = X_val_csv.astype(np.float32)

In [ ]:
input_train_data = torch.from_numpy(X_train_csv)
input_val_data   = torch.from_numpy(X_val_csv)

In [ ]:
n_epochs = 15
train_losses = []
val_losses = []

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

In [ ]:
batch_size = 128
training_set = torch.utils.data.TensorDataset(input_train, input_train_data, target_train)
train_loader = torch.utils.data.DataLoader(dataset=training_set, 
                                           batch_size=batch_size,
                                           num_workers=2,
                                           shuffle=True)
val_set = torch.utils.data.TensorDataset(input_val, input_val_data, target_val)
val_loader = torch.utils.data.DataLoader(dataset=val_set, 
                                           batch_size=batch_size,
                                         num_workers=2,
                                           shuffle=False)

In [ ]:
def binary_acc(output_pred, target):
    output_pred_tag = torch.round(torch.sigmoid(output_pred))

    correct_results_sum = (output_pred_tag == target).sum().float()
    acc = correct_results_sum/target.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [ ]:
def batch_gd(model, criterion, optimizer, train_loader, val_loader, n_epochs):
    train_losses = np.zeros(n_epochs)
    val_losses = np.zeros(n_epochs)
    train_accuracy = np.zeros(n_epochs)
    val_accuracy = np.zeros(n_epochs)
    
    
    for it in range(n_epochs):
        t0 = datetime.now()
        
        train_loss = []
        train_acc = []
        
        for inputs, inputs_data, targets in tqdm(train_loader):
            inputs, inputs_data, targets = inputs.to(device), inputs_data.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs, inputs_data)

            loss = criterion(outputs, targets)
            acc = binary_acc(outputs, targets)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            train_acc.append(acc.item())
            
        train_loss = np.mean(train_loss)
        train_acc = np.mean(train_acc)
        
        val_loss = []
        val_acc = []
        
        for inputs, inputs_data, targets in tqdm(val_loader):
            inputs, inputs_data, targets = inputs.to(device), inputs_data.to(device), targets.to(device)
            outputs = model(inputs, inputs_data)

            loss = criterion(outputs, targets)
            acc = binary_acc(outputs, targets)
            
            val_loss.append(loss.item())
            val_acc.append(acc.item())

            
        val_loss = np.mean(val_loss)
        val_acc = np.mean(val_acc)
        
        train_losses[it] = train_loss
        val_losses[it] = val_loss
        train_accuracy[it] = train_acc
        val_accuracy[it] = val_acc

        dt = datetime.now() -t0

        print(f'Epoch {it+1}/{n_epochs}, Time: {dt}, Train Loss: {train_loss:.4f}, Train_acc: {train_acc}, Val Loss: {val_loss:.4f}, Val acc: {val_acc}')
    
    return train_losses, val_losses, train_accuracy, val_accuracy

In [ ]:
train_losses, val_losses, train_accuracy, val_accuracy = batch_gd(
    model, criterion, optimizer, train_loader, val_loader, n_epochs)

In [ ]:
plt.plot(train_losses, label='train loss')
plt.plot(val_losses, label='val loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(train_accuracy, label='train accuracy')
plt.plot(val_accuracy, label='val accuracy')
plt.legend()
plt.show()

In [ ]:

n_correct_train = 0.
n_total_train = 0.
train_predictions_all = []
train_predictions_all_value = []
train_targets_all = []

for inputs, inputs_data, targets in train_loader:
    
    train_targets_np = targets.numpy()
    train_targets_all.append(train_targets_np)
    
    inputs, inputs_data, targets = inputs.to(device), inputs_data.to(device), targets.to(device)
    train_outputs = model(inputs, inputs_data)
    train_outputs = torch.sigmoid(train_outputs)
    
    _, train_predictions = torch.max(train_outputs, 1)
    train_predictions_np = train_predictions.cpu().numpy()
    train_predictions_all.append(train_predictions_np)
    
    train_predictions_np_value = train_outputs.cpu().detach().numpy()
    train_predictions_all_value.append(train_predictions_np_value)

train_predictions_np_out = []
train_predictions_np_out_value = []
train_targets_np_out = []

train_count = len(train_predictions_all)

for z in range(train_count):
    
    for a in train_predictions_all[z]:
        train_predictions_np_out.append(a)

    for a in train_predictions_all_value[z]:
        train_predictions_np_out_value.append(a)
        
    for a in train_targets_all[z]:
        train_targets_np_out.append(a)    

train_predictions_np_out = np.asarray(train_predictions_np_out)
train_predictions_np_out_value = np.asarray(train_predictions_np_out_value)
train_targets_np_out = np.asarray(train_targets_np_out)

train_count = len(train_predictions_np_out)

for z in range(train_count):
    if train_predictions_np_out[z] == np.int(train_targets_np_out[z]):
        n_correct_train += 1

train_acc = n_correct_train / train_count
# --------------------------------------------------------------------------------        
n_correct_val = 0.
n_total_val = 0.
val_predictions_all = []
val_predictions_all_value = []
val_targets_all = []
                    
for inputs, inputs_data, targets in val_loader:
    
    val_targets_np = targets.numpy()
    val_targets_all.append(val_targets_np)
    
    inputs, inputs_data, targets = inputs.to(device), inputs_data.to(device), targets.to(device)
    val_outputs = model(inputs, inputs_data)
    val_outputs = torch.sigmoid(val_outputs)
    _, val_predictions = torch.max(val_outputs, 1)
    val_predictions_np = val_predictions.cpu().numpy()
    val_predictions_all.append(val_predictions_np)

    val_predictions_np_value = val_outputs.cpu().detach().numpy()
    val_predictions_all_value.append(val_predictions_np_value)
 
val_predictions_np_out = []
val_predictions_np_out_value = []
val_targets_np_out = []
val_count = len(val_predictions_all)

for z in range(val_count):
    
    for a in val_predictions_all[z]:
        val_predictions_np_out.append(a)

    for a in val_predictions_all_value[z]:
        val_predictions_np_out_value.append(a)
        
    for a in val_targets_all[z]:
        val_targets_np_out.append(a)

val_predictions_np_out = np.asarray(val_predictions_np_out)
val_targets_np_out = np.asarray(val_targets_np_out)

val_count = len(val_predictions_np_out)

for z in range(val_count):
    if val_predictions_np_out[z] == np.int(val_targets_np_out[z]):
        n_correct_val += 1

val_acc = n_correct_val / val_count
      
print(f"Train acc: {train_acc:.4f}, Val acc: {val_acc:.4f}")

In [ ]:
fpr_train, tpr_train, _ = roc_curve(train_targets_np_out, train_predictions_np_out_value)
fpr_val, tpr_val, _ = roc_curve(val_targets_np_out, val_predictions_np_out_value)

In [ ]:
auc_pred_train = auc(fpr_train, tpr_train) 
auc_pred_val = auc(fpr_val, tpr_val) 

In [ ]:
plt.plot(fpr_train, tpr_train, label = auc_pred_train)
plt.legend()

In [ ]:
plt.plot(fpr_val, tpr_val, label = auc_pred_val)
plt.legend()